<a href="https://colab.research.google.com/github/bchuchucaa/RNNConsumoDeAgua/blob/main/RNNConsumoDeAgua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip

unzip:  cannot find or open /content/archive.zip, /content/archive.zip.zip or /content/archive.zip.ZIP.


In [1]:
from pandas import read_csv
from datetime import datetime
import pandas as pd
from matplotlib import pyplot
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import model_from_json
from sklearn.metrics import mean_squared_error
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import scale 


print('LIBRERIAS IMPORTADAS')

LIBRERIAS IMPORTADAS


**LECTURA DE DATOS**

In [19]:
#LOADING DATA 
df= read_csv('/content/AguaH.csv')
df = df.iloc[0:5000 , :]
#AJUSTE DEL FORMATO DE LAS FECHAS 
lookup = {'ENE':'01','FEB':'02','MAR':'03','ABR':'04','MAY':'05','JUN':'06','JUL':'07','AGO':'08','SEP':'09','OCT':'10','NOV':'11','DIC':'12'}
clist=[]
for col in df.columns[5:]:
    col = col[4:].split('_')
    clist.append('20'+col[1]+'-'+lookup[col[0]]+'-01')

columns = ['LANDUSE_TYPE','USER','PIPE DIAM','VENDOR','JAN16']+clist
df.columns=columns
df = df.drop(columns=['LANDUSE_TYPE','PIPE DIAM','VENDOR','JAN16'])
print("Dataframe shape ",df.shape)
#ESTE CODIGO SIRVE UNICAMENTE PARA LA INTEGRACION DE LOS NOMBRES EN EL DATAFRAME
col_list = ["Name"]
names = pd.read_csv("/content/StateNames.csv", usecols=col_list)
names = names.iloc[0:50 , :]
#ELIMINAMOS LA COLUMNA DE LOS NOMBRE YA QUE SE BINARIZARON
#df = df.iloc[: , 1:]
print("Names shape ",names.shape)
df.insert(loc=1, column='Names', value=names)

#AJUSTE DEL DATASET EN EL FORMATO SOLICITADO 
rows_list=[]
dataComp = pd.DataFrame(columns=('DATE','NOMBRE','TIPO', 'CONSUMO'))
for column in range(2,len(df.columns),1):
  selection=df.iloc[:,column]
  for row in range(len(selection)):
    df2=[df.columns[column],df.iloc[row,1],df.iloc[row,0],selection.iloc[row]]
    rows_list.append(df2)
    
dataComp = pd.DataFrame(rows_list)  


Dataframe shape  (5000, 85)
Names shape  (50, 1)


In [11]:
dataComp.head()

,0,1,2,3
0,2009-01-01,Mary,DOMESTICO MEDIO,20.0
1,2009-01-01,Annie,DOMESTICO MEDIO,NaN
2,2009-01-01,Anna,DOMESTICO RESIDENCIAL,NaN
3,2009-01-01,Margaret,DOMESTICO MEDIO,NaN
4,2009-01-01,Helen,DOMESTICO RESIDENCIAL,20.0


In [20]:
dataComp.rename(columns = {0:'DATE',1:'NOMBRE',2:'TIPO',3:'CONSUMO'}, inplace = True)
dataComp['DATE'] = pd.to_datetime(dataComp['DATE'], format='%Y-%m-%d')
dataComp=dataComp.set_index('DATE')
#RANGO DE FECHAS DE LAS MUESTRAS
print(dataComp.index.min())
print(dataComp.index.max())
#CANTIDAD DE MUESTRAS POR ANIO
print(len(dataComp['2009']))
print(len(dataComp['2010']))
print(len(dataComp['2011']))
print(len(dataComp['2012']))
print(len(dataComp['2013']))
print(len(dataComp['2014']))
print(len(dataComp['2015']))

2009-01-01 00:00:00
2015-12-01 00:00:00
60000
60000
60000
60000
60000
60000
60000


In [5]:
#DESCRIPCION DEL DATASET
dataComp.describe()

,CONSUMO
count,323059.000000
mean,94.573140
std,1032.767218
min,0.000000
25%,9.000000
50%,15.000000
75%,22.000000
max,100302.000000


In [21]:
#RELLENANDO VALORES NAN POR LA MEDIA
dataComp=dataComp.fillna(dataComp['CONSUMO'].mean())
#CONVIRTIENDO EL INDICE DEL DATAFRAME EN TIPO DATETIME
#dataComp.index = pd.to_datetime(dataComp.index)
#type(dataComp.index)
salida=dataComp["CONSUMO"]
dataComp = dataComp.drop('CONSUMO',axis = 1)
dataComp.head()

,NOMBRE,TIPO
DATE,,
2009-01-01,Mary,DOMESTICO MEDIO
2009-01-01,Annie,DOMESTICO MEDIO
2009-01-01,Anna,DOMESTICO RESIDENCIAL
2009-01-01,Margaret,DOMESTICO MEDIO
2009-01-01,Helen,DOMESTICO RESIDENCIAL


In [22]:
# Create a categorical boolean mask
categorical_feature_mask = dataComp.dtypes == object
# Filter out the categorical columns into a list for easy reference later on in case you have more than a couple categorical columns
categorical_cols = dataComp.columns[categorical_feature_mask].tolist()

# Instantiate the OneHotEncoder Object
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
dataComp['TIPO'] = dataComp['TIPO'].astype(str)
dataComp['NOMBRE'] = dataComp['NOMBRE'].astype(str)


In [23]:
# Apply ohe on data
ohe.fit(dataComp[categorical_cols])
cat_ohe = ohe.transform(dataComp[categorical_cols])

#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names(input_features = categorical_cols))
#concat with original data and drop original columns


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#df_ohe = pd.concat([dataComp, ohe_df], axis=1).drop(columns = categorical_cols, axis=1)


In [24]:
#PCA Projection to 2D
from sklearn.decomposition import PCA
num_components=5
pca = PCA(num_components)
principalComponents = pca.fit_transform(ohe_df)
a = range(num_components)
num_pc= a[::1]

principalDf = pd.DataFrame(data = principalComponents
             , columns = num_pc)

principalDf=round(principalDf, 2)
print(principalDf)

           0     1     2     3     4
0      -0.51 -0.10 -0.02  0.02  1.06
1      -0.51 -0.10 -0.02  0.02  1.04
2       0.87 -0.41 -0.05 -0.01  0.99
3      -0.51 -0.10 -0.02  0.01  1.06
4       0.87 -0.41 -0.06 -0.01  1.06
...      ...   ...   ...   ...   ...
419995 -0.52 -0.09 -0.02 -0.01 -0.01
419996 -0.52 -0.09 -0.02 -0.01 -0.01
419997 -0.52 -0.09 -0.02 -0.01 -0.01
419998  0.86 -0.39 -0.06 -0.03 -0.02
419999 -0.52 -0.09 -0.02 -0.01 -0.01

[420000 rows x 5 columns]


In [30]:
principalDf.head()

,0,1,2,3,4
0,-0.51,-0.10,-0.02,0.02,1.06
1,-0.51,-0.10,-0.02,0.02,1.04
2,0.87,-0.41,-0.05,-0.01,0.99
3,-0.51,-0.10,-0.02,0.01,1.06
4,0.87,-0.41,-0.06,-0.01,1.06


In [32]:
salida.index= principalDf.index
principalDf.insert(loc=5, column='CONSUMO', value=salida)
principalDf.rename(columns = {0:'PC1',1:'PC2',2:'PC3',3:'PC4',4:'PC5'}, inplace = True)
principalDf.head()

,PC1,PC2,PC3,PC4,PC5,CONSUMO
0,-0.51,-0.10,-0.02,0.02,1.06,20.00000
1,-0.51,-0.10,-0.02,0.02,1.04,94.57314
2,0.87,-0.41,-0.05,-0.01,0.99,94.57314
3,-0.51,-0.10,-0.02,0.01,1.06,94.57314
4,0.87,-0.41,-0.06,-0.01,1.06,20.00000


In [33]:
principalDf['CONSUMO']=principalDf['CONSUMO'].astype(float)


In [34]:
#Split en x, y
y=principalDf['CONSUMO']
dataset=principalDf.drop(['CONSUMO'], axis=1)
x=dataset
values = dataset.values
x
#-0.38, -0.16,  0.05,  0.77,  0.64

,PC1,PC2,PC3,PC4,PC5
0,-0.51,-0.10,-0.02,0.02,1.06
1,-0.51,-0.10,-0.02,0.02,1.04
2,0.87,-0.41,-0.05,-0.01,0.99
3,-0.51,-0.10,-0.02,0.01,1.06
4,0.87,-0.41,-0.06,-0.01,1.06
...,...,...,...,...,...
419995,-0.52,-0.09,-0.02,-0.01,-0.01
419996,-0.52,-0.09,-0.02,-0.01,-0.01
419997,-0.52,-0.09,-0.02,-0.01,-0.01
419998,0.86,-0.39,-0.06,-0.03,-0.02


In [35]:
# asegurarse que todo esta en formato float
values = values.astype('float32')
# normalzación
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled

array([[0.00719422, 0.24409449, 0.35294116, 0.23846154, 1.        ],
       [0.00719422, 0.24409449, 0.35294116, 0.23846154, 0.98165137],
       [1.        , 0.        , 0.33088234, 0.21538462, 0.93577987],
       ...,
       [0.        , 0.2519685 , 0.35294116, 0.21538462, 0.01834862],
       [0.99280584, 0.01574805, 0.32352942, 0.2       , 0.00917431],
       [0.        , 0.2519685 , 0.35294116, 0.21538462, 0.01834862]],
      dtype=float32)

In [36]:
#red neuronal estandar
#X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=.2)
numregistros=scaled.shape[0]
print(numregistros)

# split en conjuntos de train y test
n_train = int(round(numregistros*0.8,0)) #80% para entrenamiento

train = values[:n_train, :]
test = values[n_train:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape entrada a 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

420000
(336000, 1, 4) (336000,) (84000, 1, 4) (84000,)


In [ ]:
# Diseño de la red recurrente LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(128))
model.add(Dense(164))
model.add(Dense(32))
model.compile(loss='mae', optimizer='adam')
# entrenar red
history = model.fit(train_X, train_y, epochs=500, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Epoch 1/500
4667/4667 - 46s - loss: 0.0110 - val_loss: 0.0102 - 46s/epoch - 10ms/step
Epoch 2/500


In [ ]:
newdf=dataComp.iloc[1:2,:-1]
newdf=pd.DataFrame(newdf)


In [ ]:
principalDf.iloc[1:2,:-1]

In [ ]:
newdf['TIPO'] = newdf['TIPO'].astype(str)
newdf['NOMBRE'] =newdf['NOMBRE'].astype(str)
newdf.columns

In [ ]:
# The following code is for your newdf after training and testing on original df
# Apply ohe on newdf

cat_ohe_new = ohe.transform(newdf[categorical_cols])
#Create a Pandas DataFrame of the hot encoded column
ohe_df_new = pd.DataFrame(cat_ohe_new, columns = ohe.get_feature_names(input_features = categorical_cols))
#concat with original data and drop original columns
df_ohe_new = pd.concat([newdf, ohe_df_new], axis=1).drop(columns = categorical_cols, axis=1)


In [ ]:
pd.DataFrame(ohe_df_new)

In [ ]:
ohe_df_new.shape

In [ ]:
pred_X, pred_y = train[1:2, :], train[1:2, -1]


In [ ]:
pred_X

In [ ]:
new = pred_X.reshape((pred_X.shape[0], 1, pred_X.shape[1]))

In [ ]:
new

2009-01-01	-0.38	-0.15	0.05	0.79	0.66

In [ ]:
# predict on df_ohe_new
predict = model.predict(new)


In [ ]:
print(predict)

In [ ]:
results=model.predict(x_val)
plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(results)),results,c='r')
plt.title('validate')
plt.show()

In [ ]:
ultimosDias = dataComp['2015-11-01':'2015-12-31']
ultimosDias

In [ ]:
values = reframed.values
x_test = values[6:, :]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
x_test

In [ ]:
prediccion1SemanaDiciembre = pd.DataFrame(inverted)
prediccion1SemanaDiciembre.columns = ['pronostico']
prediccion1SemanaDiciembre.plot()
prediccion1SemanaDiciembre.to_csv('pronostico.csv')

In [ ]:
# Diseño de la red recurrente LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(128))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
model.summary()

In [ ]:
y_pred= model.predict(test_X)


In [ ]:
pd.DataFrame(test_y)

In [ ]:
y_pred.shape

In [ ]:
tes

In [ ]:
mean_squared_error(test_y, y_pred, squared=False)

In [ ]:
 # serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")
 

In [ ]:
#FUNCION PARA ESCOGER UN REGISTRO DENTRO DEL DATAFRAME PARA LAS PRUEBAS DE VALIDACION
def selectRabbit(user):
  xnew = dataComp.iloc[user , 1:999]
  return xnew


In [ ]:
#FUNCION PARA BUSCAR EL VALOR REAL DE CONSUMO DENTRO DEL DATAFRAME
def searchForReal(user):
  real= dataComp.iloc[user ,-1]
  return real

In [ ]:
user=dataComp.iloc[-1,1:]

In [ ]:
user.shape

In [ ]:
def prediccion(user):
    Xnew = np.array([dataComp.iloc[-1,1:]])#real es 79
    print(Xnew.shape)
    #xnew=selectRabbit(user)
    Xnew= scaler.transform(Xnew)
    Xnew = Xnew.reshape((Xnew.shape[0], 1, Xnew.shape[1]))
    print(Xnew.shape)
    pred= model.predict(Xnew)[0]
    return pred
    
pred=prediccion(83)
print('PREDICCION DE CONSUMO: ',pred)
#print('USUARIO CONSUMO REAL :  [',searchForReal(83),']')

In [ ]:
plt.plot(meses['2009'].values)
plt.plot(meses['2010'].values)

In [ ]:
verano2009 = dataComp['2009-01-01':'2009-01-31']
plt.plot(verano2009.values)
verano2010 = dataComp['2010-01-01':'2010-01-31']
plt.plot(verano2010.values)

In [ ]:
import matplotlib.pyplot as plt
steps = 14400
datos_train = dataComp[:-steps]
datos_test  = dataComp[-steps:]
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
ax.legend();